# Tunability for KNeighborsClassifier

# Intro

In [17]:
import pandas as pd

random_state = pd.read_csv("../../data/vars.csv")['random_state'].iloc[0]
n_iter = pd.read_csv("../../data/vars.csv")['n_iter'].iloc[0]
n_iter_BS = pd.read_csv("../../data/vars.csv")['n_iter_BS'].iloc[0]

In [18]:
df_college = pd.read_csv("../../data/processed/college.csv")
df_credit = pd.read_csv("../../data/processed/credit.csv")
df_diabetes = pd.read_csv("../../data/processed/diabetes.csv")
df_penguins = pd.read_csv("../../data/processed/penguins.csv")

In [19]:
print(df_college.info())
print()
print(df_credit.info())
print()
print(df_diabetes.info())
print()
print(df_penguins.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  will_go_to_college     1000 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 79.2+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
import numpy as np


num_pipeline = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])

col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])

model_pipeline = Pipeline(steps=[
    ('preprocessing', col_trans),
    ('model', KNeighborsClassifier())
])

# RandomizedSearchCV

In [21]:
knn_params = {
    'model__n_neighbors': range(1, 31), # paper
    'model__weights': ['uniform', 'distance'],
    #'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'model__metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
}

## Credit dataset

In [22]:
from sklearn.model_selection import RandomizedSearchCV

X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]

random_search_model = RandomizedSearchCV(estimator = model_pipeline,
                                   param_distributions = knn_params,
                                   n_iter = n_iter,
                                   cv = 5,
                                   scoring = 'f1',
                                   random_state = random_state)

random_search_model.fit(X, y)

best_f1_credit = random_search_model.best_score_

print(f"Best f1 score: {best_f1_credit}")
print(f"Best params: {random_search_model.best_params_}")

random_search_model.best_estimator_

Best f1 score: 0.8453157699330955
Best params: {'model__weights': 'uniform', 'model__n_neighbors': 7, 'model__metric': 'manhattan'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E3827D27D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E38942E450>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=7))])

## Data frame with params, results and mean result

In [23]:
tested_params = random_search_model.cv_results_['params']

In [24]:
df_knn = pd.DataFrame(tested_params)
df_knn['credit_f1'] = random_search_model.cv_results_['mean_test_score']
df_knn

,model__weights,model__n_neighbors,model__metric,credit_f1
0,distance,20,chebyshev,0.644019
1,distance,9,euclidean,0.827844
2,uniform,20,euclidean,0.834251
3,distance,20,minkowski,0.830716
4,uniform,21,chebyshev,0.615839
...,...,...,...,...
195,uniform,3,chebyshev,0.691711
196,distance,13,manhattan,0.832649
197,distance,14,chebyshev,0.659798
198,distance,2,manhattan,0.776265


## College dataset

In [25]:
from sklearn.model_selection import GridSearchCV

X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_knn['college_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_college = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_college}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.8522583786302302
Best params: {'model__metric': 'manhattan', 'model__n_neighbors': 4, 'model__weights': 'distance'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E383AE2C50>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E383AD9B10>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=4,
                                      weights='distance'))])

## Diabetes dataset

In [26]:
from sklearn.model_selection import GridSearchCV

X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_knn['diabetes_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_diabetes = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_diabetes}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.6597307060755336
Best params: {'model__metric': 'manhattan', 'model__n_neighbors': 18, 'model__weights': 'distance'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E384264F50>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E383AF2C10>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=18,
                                      weights='distance'))])

## Penguins dataset

In [27]:
from sklearn.model_selection import GridSearchCV

X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_knn['penguins_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_penguins = grid_search_model.best_score_


print(f"Best f1 score: {best_f1_penguins}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 1.0
Best params: {'model__metric': 'euclidean', 'model__n_neighbors': 17, 'model__weights': 'uniform'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E3849C0A10>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E38945C090>)])),
                ('model',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=17))])

## Tunability 

In [28]:
df_knn['mean_f1'] = df_knn[['credit_f1', 'college_f1', 'diabetes_f1', 'penguins_f1']].mean(axis=1)
df_knn

,model__weights,model__n_neighbors,model__metric,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
0,distance,20,chebyshev,0.644019,0.779271,0.595762,0.960000,0.744763
1,distance,9,euclidean,0.827844,0.832860,0.643760,0.993103,0.824392
2,uniform,20,euclidean,0.834251,0.795951,0.601064,0.993103,0.806092
3,distance,20,minkowski,0.830716,0.823631,0.637633,0.993103,0.821271
4,uniform,21,chebyshev,0.615839,0.613758,0.580452,0.957809,0.691965
...,...,...,...,...,...,...,...,...
195,uniform,3,chebyshev,0.691711,0.830688,0.586710,0.932464,0.760393
196,distance,13,manhattan,0.832649,0.816898,0.630299,0.993103,0.818237
197,distance,14,chebyshev,0.659798,0.796963,0.606307,0.960000,0.755767
198,distance,2,manhattan,0.776265,0.831953,0.566404,0.989474,0.791024


In [29]:
# sigma star
sigma_star_row = df_knn[df_knn['mean_f1'] == max(df_knn['mean_f1'])]
sigma_star_row

,model__weights,model__n_neighbors,model__metric,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
123,distance,18,manhattan,0.838871,0.826031,0.659731,0.993103,0.829434


### (3.3) Measuring Overall Tunability of a ML Algorithm

In [30]:
d_credit = -sigma_star_row['credit_f1'].iloc[0] + best_f1_credit
d_college = -sigma_star_row['college_f1'].iloc[0] + best_f1_college
d_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_f1_diabetes
d_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_f1_penguins

d = pd.Series([d_credit, d_college, d_diabetes, d_penguins])
print(d)
print(f"Mean: {d.mean()}")


0    0.006444
1    0.026227
2    0.000000
3    0.006897
dtype: float64
Mean: 0.009891984314985752


### Saving results

In [31]:
df_knn.to_csv("../../data/RSmodels/knn_results.csv", index=False)

# BayesSearchCV

In [32]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

n_iter_BS = pd.read_csv('../../data/vars.csv')['n_iter_BS'].iloc[0]
BS_iterations = pd.read_csv('../../data/vars.csv')['BS_iterations'].iloc[0]

knn_params = {
    'model__n_neighbors': Integer(1, 31),
    'model__weights': Categorical(['uniform', 'distance']),
    #'model__algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
    'model__metric': Categorical(['euclidean', 'manhattan', 'chebyshev', 'minkowski'])
}

## Credit dataset

In [33]:
X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]


bayes_search_credit_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_credit_model.fit(X, y)

best_BS_f1_credit = bayes_search_credit_model.best_score_
credit_best_estimator = bayes_search_credit_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_credit}")

credit_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['euclidean', 31, 'distance'] before, using random point ['manhattan', 20, 'uniform']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['euclidean', 31, 'distance'] before, using random point ['chebyshev', 21, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 31, 'distance'] before, using random point ['manhattan', 1, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 31, 'distance'] before, using random point ['euclidean', 16, 'distance']
  warnings.

Best f1 score: 0.8419580503889638


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E3894129D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E3896AF190>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=11))])

## College dataset

In [34]:
X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]


bayes_search_college_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_college_model.fit(X, y)

best_BS_f1_college = bayes_search_college_model.best_score_
college_best_estimator = bayes_search_college_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_college}")

college_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['chebyshev', 1, 'uniform'] before, using random point ['euclidean', 21, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 1, 'distance'] before, using random point ['euclidean', 8, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 1, 'distance'] before, using random point ['chebyshev', 15, 'uniform']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 1, 'distance'] before, using random point ['euclidean', 13, 'distance']
  warnings.warn(

Best f1 score: 0.8522583786302302


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E3887F6A10>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E389422150>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=4,
                                      weights='distance'))])

## Diabetes dataset

In [35]:
X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]


bayes_search_diabetes_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_diabetes_model.fit(X, y)

best_BS_f1_diabetes = bayes_search_diabetes_model.best_score_
diabetes_best_estimator = bayes_search_diabetes_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_diabetes}")
diabetes_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 19, 'distance'] before, using random point ['euclidean', 24, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 19, 'distance'] before, using random point ['manhattan', 3, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 17, 'distance'] before, using random point ['euclidean', 20, 'uniform']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 17, 'distance'] before, using random point ['manhattan', 20, 'distance']
  warnings.

Best f1 score: 0.6597307060755336


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E389611910>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E38B7588D0>)])),
                ('model',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=18,
                                      weights='distance'))])

## Penguins dataset

In [36]:
X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]


bayes_search_penguins_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=knn_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_penguins_model.fit(X, y)

best_BS_f1_penguins = bayes_search_penguins_model.best_score_
penguins_best_estimator = bayes_search_penguins_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_penguins}")

penguins_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 1, 'distance'] before, using random point ['euclidean', 2, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['minkowski', 1, 'distance'] before, using random point ['manhattan', 30, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 1, 'distance'] before, using random point ['euclidean', 26, 'distance']
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 1, 'distance'] before, using random point ['euclidean', 7, 'distance']
  warnings.warn

Best f1 score: 1.0


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E38B9B3990>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001E38B9B0810>)])),
                ('model', KNeighborsClassifier(n_neighbors=17))])

## Tunability

### (3.3) Measuring Overall Tunability of a ML Algorithm

In [37]:
d_BS_credit = -sigma_star_row['credit_f1'].iloc[0] + best_BS_f1_credit
d_BS_college = -sigma_star_row['college_f1'].iloc[0] + best_BS_f1_college
d_BS_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_BS_f1_diabetes
d_BS_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_BS_f1_penguins

d_BS = pd.Series([d_BS_credit, d_BS_college, d_BS_diabetes, d_BS_penguins])
print(d_BS)
print(f"Mean: {d_BS.mean()}")

0    0.003087
1    0.026227
2    0.000000
3    0.006897
dtype: float64
Mean: 0.009052554428952808


### Models saving

In [38]:
from joblib import dump

dump(bayes_search_college_model, '../../data/BSmodels/KNeighbors/bayes_search_college_model.joblib')
dump(bayes_search_credit_model, '../../data/BSmodels/KNeighbors/bayes_search_credit_model.joblib')
dump(bayes_search_diabetes_model, '../../data/BSmodels/KNeighbors/bayes_search_diabetes_model.joblib')
dump(bayes_search_penguins_model, '../../data/BSmodels/KNeighbors/bayes_search_penguins_model.joblib')

['../../data/BSmodels/KNeighbors/bayes_search_penguins_model.joblib']